In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter

import imp
import datetime
import LDA_Init as lda2
import scipy
import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import utils as my_utils
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
dataset_name = "datadf_amazon_musical"

In [3]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [4]:
dataset.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,sentiment,summary,unixReviewTime,reviewTime,cleaned,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014","[much, write, doe, exactly, supposed, filter, ...",much write doe exactly supposed filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013","[product, doe, exactly, quite, affordable, rea...",product doe exactly quite affordable realized ...


In [5]:
count_matrix, tfidf_matrix, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [6]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [7]:
barren

array([], dtype=int64)

### Making Edge_Dict

In [8]:
dataset_name

'datadf_amazon_musical'

In [ ]:
pickle_in = open("resources/docs_edges_amazon_QA_Electronics_fasttext_trained_with_stopwords_5kanswers.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [ ]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [ ]:
len(docs_edges)

In [ ]:
edge_dict = {}
for i in docs_edges:
    for j in i:
        try:
            edge_dict[j[0]] += [j[1]]
            edge_dict[j[1]] += [j[0]]
        except:
            edge_dict[j[0]] = [j[1]]
            edge_dict[j[1]] = [j[0]]

In [ ]:
for i in edge_dict.keys():
    edge_dict[i] = list(set(edge_dict[i]))

In [ ]:
len(edge_dict)

In [ ]:
lambda_param = 1.0
n_top_words = 5

In [ ]:
topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [ ]:
folder_name = "baseline_etm_amazon_fasttext_with_stopwords"

In [ ]:
folder_name = str(datetime.datetime.now()) + "_" + folder_name
os.mkdir("dumps/"+folder_name)

In [ ]:
import multiprocessing

In [ ]:
def multiprocessing_func(k):
    sampler = lda2.LdaSampler(n_topics=k, lambda_param=lambda_param)
    
    print "Running Sampler...", k
    for it, phi in enumerate(sampler.run(count_matrix, edge_dict)):
        my_utils.print_if_mod(idx, 10)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_n_topics_" + str(k))

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
pool.map(multiprocessing_func, topics_grid)

# Evaluations

In [ ]:
folder_name = "2019-08-22 00:47:11.626375_baseline_etm_amazon_fasttext_with_stopwords"

In [ ]:
def multiprocessing_func_eval(k):
    sampler = joblib.load("dumps/"+folder_name + "/sampler_n_topics_" + str(k))

    t_words = sampler.getTopKWords(n_top_words, words)
    top_words = [t_words[i] for i in t_words.keys()]
    dt_distribution = sampler.theta()
    
    print
    print "Topics:", k
    print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
    print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
    print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [ ]:
topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [ ]:
for k in topics_grid:
    multiprocessing_func_eval(k)

# TSNE

In [ ]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()